# Week 6- MapReduce and Apache Spark

**Objectives**: Today we are going to work with Apache Spark on AWS. Spark is one of the most popular "big data" platforms and supports both batch processing like Hadoop and streaming. We will also explore the basics of the MapReduce programming model that was the basis for Apache Hadoop. Today we will:
  
* Review MapReduce
* Review the conceptual foundation of MapReduce with parallel Python
* Review Spark and its place in the "big data" technology ecosystem
* Set up our Spark environment including PySpark
* Read data from S3 into an RDD
* Conduct some analyses using Spark

# MapReduce and Apache Hadoop

<img src="https://raw.githubusercontent.com/azbones/big_data/master/images/map_reduce.png">
(source: http://static.googleusercontent.com/media/research.google.com/en//archive/mapreduce-osdi04.pdf)

According to the Wall Street Journal:

>Researcher Gartner Inc. says Hadoop adoption [remains low](http://blogs.wsj.com/cio/2015/05/13/hadoop-corporate-adoption-remains-low-gartner/) as firms struggle to articulate Hadoop’s business value and overcome a shortage of workers who have the skills to use it. A survey of 284 global IT and business leaders in May found more than half had no plans to invest in Hadoop. Adoption could grow with the use of [tools based on SQL](http://blogs.wsj.com/cio/2015/03/31/corporate-hadoop-adoption-is-growing-barclays-report-says/), a query language that corporate IT shops know well, Barclays analyst Raimo Lenschow said earlier this year.

(source: http://blogs.wsj.com/cio/2015/12/11/cio-explainer-what-is-hadoop/)

# Serial Execution Through One Process

To demonstrate the conceptual foundations of MapReduce, we will use TextBlob to count noun phrase using a single-process serial approach and then use the IPython parallel library to conduct the same analysis using several workers in parallel. We will use the IPython magic command <code>%time</code> to capture the time of execution for each version.

In [3]:
from textblob import TextBlob
import codecs
with codecs.open('/Users/sellco/Documents/My_Documents/ASU/big_data/week_6/datasets/moby_full','r',encoding='utf8') as f:
    text = f.read()

In [4]:
full_text = TextBlob(text)

In [5]:
full_text

TextBlob("﻿The Project Gutenberg EBook of Moby Dick; or The Whale, by Herman Melville

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org


Title: Moby Dick; or The Whale

Author: Herman Melville

Last Updated: January 3, 2009
Posting Date: December 25, 2008 [EBook #2701]
Release Date: June, 2001

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK MOBY DICK; OR THE WHALE ***




Produced by Daniel Lazarus and Jonesey





MOBY DICK; OR THE WHALE

By Herman Melville




Original Transcriber's Notes:

This text is a combination of etexts, one from the now-defunct ERIS
project at Virginia Tech and one from Project Gutenberg's archives. The
proofreaders of this version are indebted to The University of Adelaide
Library for preserving the Virginia Tech version. The resulting etext
was 

In [6]:
%time serial = full_text.np_counts

CPU times: user 1min 40s, sys: 1.02 s, total: 1min 41s
Wall time: 1min 41s


In [7]:
print 'Length of noun phrases is {}'.format(len(serial))
print 'Sum of noun phrase counts is {}'.format(sum(serial.values()))

Length of noun phrases is 4720
Sum of noun phrase counts in 6850


# Parallel Execution Through Four Worker Processes

For the next example, we are going to use IPython Parrallel to conduct the same analysis using a process similar to that of MapReduce. While we will be executing this on our individual computers, the same code with minor changes could be used across different physical devices. 


from command line start iPython nodes:

<code>ipcluster start -n 4</code>

In [1]:
# Start IPython Parallel in notebook and check for workers

from ipyparallel import Client
c = Client()
print 'These are the worker ids:{}'.format(c.ids)

These are the worker ids:[0, 1, 2, 3]


In [2]:
# Assign all workers to a view

dview=c[:]

In [8]:
text_list = ['moby25a', 'moby25b', 'moby25c', 'moby25d']

In [9]:
@dview.parallel(block=True)
def read_texts_parallel(text):
    from textblob import TextBlob
    import codecs
    with codecs.open('/Users/sellco/Documents/My_Documents/ASU/big_data/week_6/datasets/{}'.format(text[0]),'r',encoding='utf8') as f:
        text = f.read()
    blob = TextBlob(text)
    counts = blob.np_counts
    return dict(counts)    

In [10]:
from collections import Counter

def map_reduce(texts):
    # This effectively maps the iterable list of texts to the function on each worker
    mapped_text = read_texts_parallel(texts)
    # This takes the returned map results and combines them in the notebook process
    reduced = reduce(lambda x, y:Counter(x)+Counter(y), mapped_text)
    return reduced


%time map_reduced = map_reduce(text_list)

CPU times: user 67.1 ms, sys: 5.28 ms, total: 72.4 ms
Wall time: 9 s


In [11]:
print 'Length of noun phrases is {}'.format(len(map_reduced))
print 'Sum of noun phrase counts in {}'.format(sum(map_reduced.values()))

Length of noun phrases is 4719
Sum of noun phrase counts in 6850


In [12]:
set(serial).difference(set(map_reduced))

{u'great battle wherein'}